In [78]:
from tqdm import tqdm_notebook as tqdm

#from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import train_test_split
import pickle

import nltk
import joblib
from nltk.corpus import stopwords
from pymystem3 import Mystem
import re

from os.path import join
from glob import glob

from time import time
import pandas as pd
import numpy as np
import pickle




In [79]:
def preprocessing_texts(df):
    df = df['text']
    df = re.sub(r'\(http[\S]*\)', '', df)
    df = re.sub(r'\n', '', df)
    df = re.sub(r'\[', '', df)
    df = re.sub(r'\]', '', df)
    return df

In [80]:
df_texts = pd.read_csv('tj_texts_upd_large.csv')

In [81]:
df_texts

,doc_id,text,preprocessed
0,3,Последняя версия мессенджера Whats App на And...,Последняя версия мессенджера Whats App на And...
1,4,Подросток Марсель Фернандес Фильо из Бразилии...,Подросток Марсель Фернандес Фильо из Бразилии...
2,12,"Приставка сделана в виде небольшой флешки, ко...","Приставка сделана в виде небольшой флешки, ко..."
3,15,Популярный ресурс про Apple и тренды перезапу...,Популярный ресурс про Apple и тренды перезапу...
4,16,"Житель Ижевска, который разместил у себя на с...","Житель Ижевска, который разместил у себя на с..."
...,...,...,...
118657,449964,Встречаются 5ый член и Кнорозов. Кнорозов расс...,Встречаются 5ый член и Кнорозов. Кнорозов расс...
118658,449971,Кнорозов пишет объявление: “Админское собрание...,Кнорозов пишет объявление: “Админское собрание...
118659,449983,искусственный интеллект в конечном счете приде...,искусственный интеллект в конечном счете приде...
118660,449990,"Какие сервисы затронуло, предварительные причи...","Какие сервисы затронуло, предварительные причи..."


In [5]:
#df_texts['preprocessed'] = df_texts.apply(preprocessing_texts, axis=1)

In [82]:
with open('./stopwords/stopwords_updated.pkl', 'rb') as f:
    stopwords = pickle.load(f)

In [83]:
#idx = np.random.permutation(df_texts.shape[0])

In [84]:
#with open('idx_upd.npy', 'wb') as f:
#    np.save(f, idx)

In [85]:
texts_train = list(df_texts['preprocessed'][idx[:round(0.7 * df_texts.shape[0])]])
texts_test = list(df_texts['preprocessed'][idx[round(0.7 * df_texts.shape[0]):]])

In [86]:
#with open('./texts_test', 'rb') as f:
#    texts_test = pickle.load(f)
#with open('./texts_train', 'rb') as f:
#    texts_train = pickle.load(f)

In [106]:
tf_idf = TfidfVectorizer(input='content',
                             stop_words=stopwords,
                             smooth_idf=False
                         )
tf_idf.fit(texts_train)
#getting idfs
idfs = tf_idf.idf_
#sorting out too rare and too common words
#original 1.3 and 7
# 2 6
lower_thresh = 1
upper_thresh = 7.5
not_often = idfs > lower_thresh
not_rare = idfs < upper_thresh

mask = not_often * not_rare

good_words = np.array(tf_idf.get_feature_names())[mask]
#deleting punctuation as well.
cleaned = []
for word in good_words:
    word = re.sub("^(\d+\w*$|_+)", "", word)
    if len(word) == 0:
        continue
    cleaned.append(word)
print("Len of original vocabulary: %d\nAfter filtering: %d"%(idfs.shape[0], len(cleaned)))

Len of original vocabulary: 541087
After filtering: 16727


In [107]:
#cleaned

['abc',
 'ai',
 'air',
 'airbnb',
 'airlines',
 'airpods',
 'aliexpress',
 'amazon',
 'america',
 'american',
 'android',
 'ap',
 'api',
 'app',
 'apple',
 'ar',
 'art',
 'associated',
 'awards',
 'baza',
 'bbc',
 'bell',
 'big',
 'black',
 'bloomberg',
 'blue',
 'bluetooth',
 'bmw',
 'boeing',
 'bros',
 'business',
 'buzzfeed',
 'call',
 'capital',
 'cbs',
 'cdx',
 'change',
 'china',
 'chrome',
 'city',
 'class',
 'club',
 'cnbc',
 'cnn',
 'co',
 'company',
 'covid',
 'covid19',
 'daily',
 'dark',
 'data',
 'david',
 'day',
 'dc',
 'de',
 'deep',
 'delivery',
 'digital',
 'disney',
 'dragon',
 'dtf',
 'dvd',
 'ebay',
 'entertainment',
 'face',
 'facebook',
 'falcon',
 'fi',
 'financial',
 'fm',
 'forbes',
 'foundation',
 'fox',
 'free',
 'future',
 'galaxy',
 'game',
 'games',
 'get',
 'github',
 'gizmodo',
 'global',
 'gmail',
 'go',
 'good',
 'google',
 'gps',
 'group',
 'gta',
 'guardian',
 'hbo',
 'hd',
 'hollywood',
 'home',
 'house',
 'html',
 'huawei',
 'human',
 'ibm',
 'id',

In [108]:
m = Mystem()
stemmed = set()
voc_len = len(cleaned)
for i in tqdm(range(voc_len)):
    word = cleaned.pop()
    stemmed_word = m.lemmatize(word)[0]
    stemmed.add(stemmed_word)
    
stemmed = list(stemmed)
print('After stemming: %d'%(len(stemmed)))

/var/folders/h9/f5kwlqld0wn87p77fb7zxcpc0000gn/T/ipykernel_43220/3226745388.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(voc_len)):


  0%|          | 0/16727 [00:00<?, ?it/s]

After stemming: 5938


In [109]:
#training count vectorizer
voc = {word : i for i,word in enumerate(stemmed)}

count_vect = CountVectorizer(input='content',
                             stop_words=stopwords,
                             vocabulary=voc)

dataset = count_vect.fit_transform(texts_train)

In [110]:
 #training LDA
lda = LDA(n_components = 60, max_iter=30, n_jobs=6, learning_method='batch', verbose=1)
lda.fit(dataset)

iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30


LatentDirichletAllocation(max_iter=30, n_components=60, n_jobs=6, verbose=1)

In [111]:
joblib.dump(lda, './models/lda.pkl')
joblib.dump(count_vect, './models/countVect.pkl')
joblib.dump(tf_idf,'./models/tf_idf.pkl')

['./models/tf_idf.pkl']

In [24]:
#with open('texts_train', 'wb') as fp:
#    pickle.dump(texts_train, fp)

In [23]:
#with open('texts_test', 'wb') as fp:
#    pickle.dump(texts_test, fp)